# Project 3 - Open Street Map Data Wrangling

## Abstract
Open Street Map is a project to create a open source mapping of the world. It could most easily be understood as the Wikipedia of maps, where anyone in the world can add to the mapping dataset. The data provides a rich dataset, both interesting in the context of learning more about various geographic areas around the world, or from the data itself, such as how many people contributed to the dataset for a particular area. With any project reliant on human input though the data is sometimes inconsistent. 

### Los Angeles California
Los Angeles California was chosen for two primary reasons. One is that Los Angeles is largest city in America, and the second most populous. It is very likely that there are more records in Los Angeles than most other American cities and it is also likely that there are more contributers. The second reason is that the I was born close to Los Angeles so to me it is a more interesting dataset.

## Methodology
The map data was initially downloaded from MapZen

## Data Munging

### Problematic Tags

In [2]:
from programs import tags

file_location = "data\los-angeles_california.osm"

tag = tags.process_map(file_location)

In [4]:
tag

{'lower': 1804949, 'lower_colon': 2122407, 'other': 176260, 'problemchars': 0}

Out of the 4,103,616 tags luckily none of them contain any of characters labeled as problem characters in the Lesson 6 example.

### Number of contributors

In [7]:
from programs import users
users = users.process_map(file_location)

In [9]:
len(users)

3026

Out of the millions of people that have visited or live in LA it seems that only 3026 people are responsible for all the points in the LA Open Street Map Project. Further analysis will be done after the MongoDB database has been created.

### Create json
For import into Mongodb the osm file, an XML type file, will be converted into a json using python. During the conversion process street names are checked and converted. The list of conversations was generated manually by reviewing a list of all street suffixes to check for repeats or typos.

In [ ]:
from programs import data

data.process_map(file_location)

# Appendix
Map Source - https://mapzen.com/data/metro-extracts